In [3]:
from module.lastdance import Algorithm
from module.jh import JH

In [4]:
jh = JH()

In [5]:
import pandas as pd 
from kiwipiepy import Kiwi
import numpy as np
import re
import pickle

In [6]:
with open(r"module/nnp_list.pkl", "rb") as r :
    nnp_list = pickle.load(r)

with open(r"module/WD.StandardDict.p", "rb") as r :
    StandradDict = pickle.load(r)

with open(r"module/WD.WordToStandard.p", "rb") as r :
    WordToStandard = pickle.load(r)

### Class process code re-factoring

In [7]:
class Algorithm:
    def __init__(self) -> None:
        self.stop_words = ["정식","안주"] 
        self.option_list = ["추가","선택", "많이", "적게", "조금","빼고", "리필","tall","small","large","xl","인분","l","m","s","grande","venti"
                            "reguler", "기본"]


    # stop words algorithm / Step 1 #
    def is_stop_words(self, stop_word_text:str, stop_words_param:list) -> str:
        """_summary_

        Args:
            stop_word_text (str): _description_
            stop_words_param (list): _description_

        Returns:
            str: _description_
        """
        result = jh.kiwi.space(stop_word_text).split(" ")
        for i in result :
            if i in stop_words_param :
                pat = re.compile(f"{i}")
                stop_word_text = re.sub(pat, "", stop_word_text)
        return stop_word_text.strip()
    # apply 선언 함수
    stop_words_func = lambda self, txt : self.is_stop_words(txt, self.stop_words)
    
    # for bascket data algorithm / Step 2 #
    def bascket_remove(self, text:str) -> str :
        """_summary_

        Args:
            text (str): _description_

        Returns:
            str: _description_
        """
        pat = re.compile(r"\([^)]*\)|\[[^)]*\]")
        pat2 = re.compile(r'\(|\)|\[|\]')
        bascket_find = re.findall(pat, text)
        if bascket_find : 
            res = [pat2.sub("",i) for i in bascket_find ]
            for i in res :
                try : #  있는 경우 고유명사 검사
                    if nnp_list[nnp_list.index(i)] :
                        print("1") 
                        try_pattern = re.compile(f"\({i}\)|\[{i}\]")
                        sub_result = try_pattern.sub("", text)  
                    return "".join(sub_result)   
                except : # 없는 경우 옵션 검사
                    if i in self.option_list :
                        print(i)
                        except_pattern = re.compile(f"\({i}\)|\[{i}\]")
                        except_sub_result = except_pattern.sub("", text)
                        print(except_sub_result)

                    if except_sub_result :
                        return "".join(except_sub_result)
            return text

    # option check algorithm / Step 3 #
    def is_option(self, option_text:str, option_li:list) ->str:
        """_summary_

        Args:
            option_text (str): _description_
            option_li (list): _description_

        Returns:
            str: _description_
        """
        result = jh.kiwi.space(option_text).split(" ")
        for i in result :
            if i in option_li :
                pat = re.compile(f"{i}")
                valiable = "".join(pat.findall(option_text))
                return " ".join(result), f"옵션 {valiable}"
        else :
            return option_text, ""
    # apply 선언 함수
    option_func = lambda self, txt : self.is_option(txt, self.option_list)

    # special_character algorithm / Step 4 #
    def special_char(self, special_text:str) -> str: 
        """_summary_

        Args:
            special_text (str): _description_

        Returns:
            str: _description_
        """
        split_result = re.split(r"\+|\&|\/|\,", special_text)
        if len(split_result) >= 2 :
            combine_text = [split_result[split_result.index(i)].strip() for i in split_result]
            pattern = re.compile(r"[^\w\s]+")
            return [pattern.sub("", i ) for i in combine_text]
        else : 
            pattern = re.compile(r"[^\w\s]+")
            return pattern.sub("", special_text)

    # unit algorithm / Step 5 #
    extract_morph = lambda self, x : jh.morph(x)

    # space algorithm / Step 6 #
    spacing_text = lambda self, x : jh.kiwi.space(x)

    # remove_nnp/ Step 7 #
    def nnp_extract(self, text:str, nnp_list:list) -> str :
        """_summary_

        Args:
            text (str): _description_
            nnp_list (list): _description_

        Returns:
            str: _description_
        """
        res = []
        for i in text.split() :      
            if i not in nnp_list :
                res.append(i)
        return " ".join(res)

    nnp_remove = lambda self, x: self.nnp_extract(x, nnp_list)


    # std words / Step 8 #
    def word_standard(self,txt:str) -> str :
        """_summary_

        Args:
            txt (str): _description_

        Returns:
            str: _description_
        """
        res = []
        for word in txt.split(' '):
            try:
                res.append(WordToStandard[word])
            except :
                res.append(word)
        return " ".join(res)
        
    # data standard / Step 9 #
    def member_menu(self, txt:str) -> str :
        """_summary_

        Args:
            txt (str): _description_

        Returns:
            str: _description_
        """
        score_ls = []
        word_ls = txt.split(' ')
        for word in word_ls:
            try:
                score_ls.append(StandradDict[word]['mean'])
            except:
                score_ls.append(10)
        menu = word_ls[np.array(score_ls).argmin()]
        word_ls.pop(word_ls.index(menu))
        return word_ls, menu



In [8]:
df = pd.read_excel(r"C:\Users\user\Desktop\workbench\dataset\sampleTest.xlsx")

In [17]:
train = pd.read_excel(r"C:\Users\user\Desktop\workbench\dataset\offerTrain.xlsx")

In [9]:
algo = Algorithm()

### 파이프라인 적용 할 프로세스형 함수

In [10]:
def preprocessing(text) :
    res = {'메뉴':None, '맛/재료':None, 
        '단위':None, '세트/옵션':None}
        
    x = algo.stop_words_func(text) # 불용어 알고리즘
    # print("불용어 처리 알고리즘 : ", x)
    x = algo.bascket_remove(x)
    # print("괄호 처리 알고리즘 : ", x )
    x = algo.option_func(x) # 옵션 알고리즘
    # print("옵션 처리 알고리즘 : ", x)
    if x[-1] : # 옵션 알고리즘을 통해 옵션 데이터로 판별 될 경우 함수 종료 
        res["세트/옵션"] = "옵션"
        res["메뉴"] = x[0]
        return res
    x = algo.special_char(x[0]) # 특수문자 알고리즘 
    # print("특수문자 처리 알고리즘 :", x)
    if type(x) == list  : # (세트를 의미하는) 특수문자가 존재 하는 경우
        res["세트/옵션"] = "세트"
        return res
    rs = jh.morph(x)
    print(rs)
    # print("단위 처리 알고리즘 : ", rs )
    try : 
        if rs[-1] :
            res["단위"] = "".join(rs[-1]) 
        x = algo.spacing_text(rs[0]) # 스페이싱 알고리즘
    # print("스페이싱 알고리즘 : ", x )
    except TypeError as e:
        pass
    x = algo.nnp_remove(x)
    # print("고유명사 제거 알고리즘 : ", x)
    x = algo.word_standard(x)
    # print("단어 표준화 알고리즘 : ", x)
    output = algo.member_menu(x)
    # print("데이터 표준화 알고리즘 : ", output) 
    res["맛/재료"] = ",".join(output[0]); res["메뉴"] = output[1]

    ls = [i.form for i in jh.kiwi.tokenize(x) if i.tag[0] == "N"]

    return {"menu" : " ".join(ls),
            "std" : res}


In [9]:
from sklearn.pipeline import Pipeline

In [10]:
pipeline = Pipeline([])

TypeError: __init__() missing 1 required positional argument: 'steps'

In [10]:
preprocessing("맛있는김치찌개")

--- Logging error ---
--- Logging error ---
--- Logging error ---
--- Logging error ---
--- Logging error ---
Error in sys.excepthook:
Traceback (most recent call last):
  File "c:\Users\user\anaconda3\lib\traceback.py", line 495, in __init__
    context = TracebackException(
  File "c:\Users\user\anaconda3\lib\traceback.py", line 495, in __init__
    context = TracebackException(
  File "c:\Users\user\anaconda3\lib\traceback.py", line 495, in __init__
    context = TracebackException(
  [Previous line repeated 13 more times]
  File "c:\Users\user\anaconda3\lib\traceback.py", line 486, in __init__
    exc_value.__cause__.__traceback__,
AttributeError: 'str' object has no attribute '__traceback__'

Original exception was:
TypeError: print_exception(): Exception expected for value, str found

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 345

: 

: 

In [27]:
for_apply = lambda x : preprocessing(x)["menu"]

In [28]:
for_apply("맛있는김치찌개")

--- Logging error ---
--- Logging error ---
--- Logging error ---
--- Logging error ---
--- Logging error ---
Error in sys.excepthook:
Traceback (most recent call last):
  File "c:\Users\user\anaconda3\lib\traceback.py", line 495, in __init__
    context = TracebackException(
  File "c:\Users\user\anaconda3\lib\traceback.py", line 495, in __init__
    context = TracebackException(
  File "c:\Users\user\anaconda3\lib\traceback.py", line 495, in __init__
    context = TracebackException(
  [Previous line repeated 13 more times]
  File "c:\Users\user\anaconda3\lib\traceback.py", line 486, in __init__
    exc_value.__cause__.__traceback__,
AttributeError: 'str' object has no attribute '__traceback__'

Original exception was:
TypeError: print_exception(): Exception expected for value, str found

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 345

: 

: 

In [14]:
df["상품명"].apply(for_apply)

array([''], dtype=object)

### 고유명사 구축

In [18]:
sido = pd.read_csv(r"C:\Users\user\Desktop\workbench\dataset\국토교통부_전국 법정동_20221031.csv", encoding= 'cp949')

In [19]:
world = pd.read_csv(r"C:\Users\user\Desktop\workbench\dataset\외교부_국가(지역)별 일반현황_20220629.csv", encoding= 'cp949')

In [20]:
df_list = df[df["상점명"].str.contains("_")]["상점명"].str.split("_").str.get(0).unique().tolist() # 테스트 기업 명

In [21]:
train_list = train[train["상점명"].str.contains("_")]["상점명"].str.split("_").str.get(0).unique().tolist()

In [22]:
sigungu_list = sido["시군구명"].str[:-1].unique().tolist()

In [23]:
myundong_list = sido["시도명"].str[:2].unique().tolist()

In [24]:
world_list = world["한글 국가명"].tolist()

In [25]:
nnp_list = world_list + myundong_list + sigungu_list + train_list + df_list

In [27]:
len(nnp_list)

894

In [28]:
with open("nnp_list.pkl", "wb") as f :
    pickle.dump(nnp_list, f)

In [40]:
len(nnp_list)

894

### 괄호 데이터 처리 로직 구현

In [101]:
def bascket_remove(text) :
    pat = re.compile(r"\([^)]*\)|\[[^)]*\]")
    pat2 = re.compile(r'\(|\)|\[|\]')
    # result = re.sub(pat2, '', text)]
    bascket_find = re.findall(pat, text)
    res = [pat2.sub("",i) for i in bascket_find ]
    for i in res :
        try : 
            if nnp_list[nnp_list.index(i)] : 
                pattern = re.compile(f"\({i}\)|\[{i}\]")
                delete = pattern.sub("", text)     
        except :
            if i in algo.option_list :
                pattern = re.compile(f"\({i}\)|\[{i}\]")
                delete = pattern.sub("", delete)
    return delete


In [102]:
bascket_remove("[커피에반하다]녹차라떼ice(tall)")

'녹차라떼ice'

In [31]:
nnp_list[nnp_list.index("커피에반하다")]

'커피에반하다'

In [40]:
def bascket_remove(text:str) :
    """_summary_

    Args:
        text (str): _description_

    Returns:
        str: _description_
    """
    pat = re.compile(r"\([^)]*\)|\[[^)]*\]")
    pat2 = re.compile(r'\(|\)|\[|\]')
    bascket_find = re.findall(pat, text)
    print(bascket_find)
    if bascket_find : 
        res = [pat2.sub("",i) for i in bascket_find ]
        print(res)
        for i in res :
            try : #  있는 경우 고유명사 검사
                print("??")
                if nnp_list[nnp_list.index(i)] : 
                    print("if 안")
                    pattern = re.compile(f"\({i}\)|\[{i}\]")
                    sub_result = pattern.sub("", text)  
                return "".join(sub_result)   
            except : # 없는 경우 옵션 검사
                print("except")
                if i in algo.option_list :
                    pattern = re.compile(f"\({i}\)|\[{i}\]")
                    except_sub_result = pattern.sub("", text)
                if except_sub_result :
                    return "".join(except_sub_result)
    return text

In [41]:
bascket_remove("아메리카노1L")

[]


'아메리카노1L'